In [1]:
import sys
sys.path.append("/home/kelab/m6AAIpy2")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('/home/kelab/Desktop/iM6A/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(y, custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for y in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
models

### Read data

In [6]:
Fasta = pd.read_csv("Temp/mm10_LastIntron_Fasta.csv")

In [8]:
Fasta = Fasta[Fasta["exonCount"]>=3]
Fasta = Fasta.reset_index(drop = True)

### Select positive strand

In [10]:
Fasta_Pos = Fasta[Fasta["strand"]=="+"]

In [11]:
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [14]:
Sequence = Fasta_Pos["Sequence"].tolist()

In [ ]:
df = pd.DataFrame(range(-1000,1001),columns=["index"])

In [80]:
for i in range(len(Fasta_Pos)):
    
    # Define exon location   
    Exon = []
    a = Fasta_Pos.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Pos.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)    
    
    ## Define sequence of exon and intron
    input_sequence = Sequence[i]
    First = [input_sequence[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        seq = input_sequence[(CumSum[m-1]):(CumSum[m])]
        First.append(seq)
    
    # Define sequence of exons
    ExonSeq = []
    for n in range(0,len(First),2):
        seq = First[n]
        ExonSeq.append(seq)
   
    # Define sequence of introns   
    IntronSeq = []
    for n in range(1,len(First),2):
        seq = First[n]
        if len(seq)>=400:
            seq = seq[0:200] + seq[-200:]
        if len(seq)<400:
            seq = seq
        IntronSeq.append(seq)
    
    # Define length of exons
    ExonLength = []
    for t in ExonSeq:
        ExonLength.append(len(t))

    # Define length of introns
    IntronLength = []
    for t in IntronSeq:
        IntronLength.append(len(t))
    IntronLength.append(0)
    IntronSeq.append("")
    
    # Define exon and intron length in transcript
    Length = []
    for t in range(0,len(ExonLength),1):
        length1 = ExonLength[t]
        length2 = IntronLength[t]
        Length.append(length1)        
        Length.append(length2)
    Length.pop()
    
    # Define transcript
    Transcript = ""
    for t in range(0,len(ExonSeq),1):
        seq1 = ExonSeq[t]
        seq2 = IntronSeq[t]
        Transcript = Transcript + seq1 + seq2

    ## Predict truncated transcript
    input_sequence = Transcript
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()
    
    # Define prob in exons
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)    
    
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)    
 
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-1):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Pos.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)        

### Select negative strand

In [16]:
Fasta_Neg = Fasta[Fasta["strand"]=="-"]

In [17]:
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [18]:
Sequence = Fasta_Neg["Sequence"].tolist()

In [ ]:
for i in range(len(Fasta_Neg)):
    
    # Define exon location
    Exon = []
    a = Fasta_Neg.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Neg.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    Length = Length[::-1]
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)
   
    ## Define sequence of exon and intron
    input_sequence = Sequence[i]
    First = [input_sequence[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        seq = input_sequence[(CumSum[m-1]):(CumSum[m])]
        First.append(seq)
    
    # Define sequence of exons
    ExonSeq = []
    for n in range(0,len(First),2):
        seq = First[n]
        ExonSeq.append(seq)
   
    # Define sequence of introns   
    IntronSeq = []
    for n in range(1,len(First),2):
        seq = First[n]
        if len(seq)>=400:
            seq = seq[0:200] + seq[-200:]
        if len(seq)<400:
            seq = seq
        IntronSeq.append(seq)
    
    # Define length of exons
    ExonLength = []
    for t in ExonSeq:
        ExonLength.append(len(t))

    # Define length of introns
    IntronLength = []
    for t in IntronSeq:
        IntronLength.append(len(t))
    IntronLength.append(0)
    IntronSeq.append("")
    
    # Define exon and intron length in transcript
    Length = []
    for t in range(0,len(ExonLength),1):
        length1 = ExonLength[t]
        length2 = IntronLength[t]
        Length.append(length1)        
        Length.append(length2)
    Length.pop()
    
    # Define transcript
    Transcript = ""
    for t in range(0,len(ExonSeq),1):
        seq1 = ExonSeq[t]
        seq2 = IntronSeq[t]
        Transcript = Transcript + seq1 + seq2
    
    ## Predict truncated transcript
    input_sequence = Transcript
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()    
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)

    # Define prob in exons
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)
    
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-1):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Neg.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)            

In [ ]:
df

In [ ]:
df.drop(["index"], axis=1, inplace=True)
df = df.T

In [ ]:
df.loc["Sum"] = df.sum()
df = df.T

In [ ]:
Value = df[["Sum"]]

In [ ]:
Value.head(5)

In [ ]:
df.drop(["Sum"], axis=1, inplace=True)
df = df.T

In [ ]:
df.loc["Number"] = (df > 0).sum()
df = df.T

In [ ]:
Number = df[["Number"]]

In [ ]:
Number.head(5)

In [ ]:
Data = pd.concat([Value, Number], axis=1)

In [ ]:
Data["Mean"] = Data["Sum"]/Data["Number"]

In [ ]:
Data = Data.head(2000)
Sum = []
Number = []

for j in range(0,len(Data),10):

    a = Data.loc[j:(j+9),"Sum"]
    b = np.mean(a, axis=0)
    Sum.append(b)
    
    a = Data.loc[j:(j+9),"Number"]
    b = np.mean(a, axis=0)
    Number.append(b)

Result = pd.DataFrame({"Index":range(-1000,1000,10), "Sum":Sum, "Number":Number})
Result["Mean"] = Result["Sum"]/Result["Number"] 

In [ ]:
Result.to_csv("Mouse_iM6A_LastExonStart_10interval_IntronTruncation.csv", index=0)